# Gas Rate Structure
https://www.pge.com/assets/rates/tariffs/grf1024.pdf

In [22]:
# PG&E October 2024 Gas Rate Structure

# Baseline Allowance for Residential Gas Rates (in therms/day)
baseline_allowances = {
    "PGE": {
        "G-1": {
            "territories": {
                "P": {
                    "summer": 0.39,  # therms/day
                    "winter_offpeak": 1.88,
                    "winter_onpeak": 2.19,
                },
                "Q": {
                    "summer": 0.56,
                    "winter_offpeak": 1.48,
                    "winter_onpeak": 2.00,
                },
                "R": {
                    "summer": 0.36,
                    "winter_offpeak": 1.24,
                    "winter_onpeak": 1.81,
                },
                "S": {
                    "summer": 0.39,
                    "winter_offpeak": 1.38,
                    "winter_onpeak": 1.94,
                },
                "T": {
                    "summer": 0.56,
                    "winter_offpeak": 1.31,
                    "winter_onpeak": 1.68,
                },
                "V": {
                    "summer": 0.59,
                    "winter_offpeak": 1.51,
                    "winter_onpeak": 1.71,
                },
                "W": {
                    "summer": 0.39,
                    "winter_offpeak": 1.14,
                    "winter_onpeak": 1.68,
                },
                "X": {
                    "summer": 0.49,
                    "winter_offpeak": 1.48,
                    "winter_onpeak": 2.00,
                },
                "Y": {
                    "summer": 0.72,
                    "winter_offpeak": 2.22,
                    "winter_onpeak": 2.58,
                }
            }
        }
    }
}

# Rate plans for gas (per therm)
rate_plans = {
    "PGE": {
        "G-1": {
            "baseline": {
                "procurement_charge": 0.35402,  # per therm
                "transportation_charge": 1.94995,  # per therm
                "total_charge": 2.30397,  # per therm
            },
            "excess": {
                "procurement_charge": 0.35402,  # per therm
                "transportation_charge": 2.44371,  # per therm
                "total_charge": 2.79773,  # per therm
            }
        },
        "G-PPPS": {  # Public Purpose Program Surcharge
            "residential": 0.11051,  # per therm
        },
        "G-NT": {
            "procurement_charge": 0.27473,
            "transportation_charge": 1.30455,  # based on usage tiers
            "total_charge": 1.57928  # per therm (example tier)
        }
    }
}

In [29]:
import pandas as pd
from datetime import datetime, timedelta

scenario = "scenario-baseline"
housing_type = "single-family-detached"

csv_file_path = f"alameda/{scenario}/{housing_type}/step4_natural_gas_loads.csv"
load_profile_df = pd.read_csv(csv_file_path)

load_profile_df.head()


,timestamp,out.natural_gas.heating.energy_consumption,out.natural_gas.hot_water.energy_consumption,out.natural_gas.range_oven.energy_consumption,out.natural_gas.clothes_dryer.energy_consumption,out.natural_gas.fireplace.energy_consumption,calculated.natural_gas.total.kwh,calculated.natural_gas.total.therms,month
0,2018-01-01 00:15:00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1
1,2018-01-01 00:30:00,0.0,0.003239,0.0,0.0,0.0,0.003239,0.000111,1
2,2018-01-01 00:45:00,0.0,0.003411,0.0,0.0,0.0,0.003411,0.000116,1
3,2018-01-01 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1
4,2018-01-01 01:15:00,0.0,0.003104,0.0,0.0,0.0,0.003104,0.000106,1


In [47]:
load_profile_df['calculated.natural_gas.total.therms'].sum()

456.9383758888757

In [65]:
def categorize_season(month_number):
    """Categorize a given timestamp into Winter Off-Peak, Winter On-Peak, or Summer."""
    
    if month_number in [11, 2, 3]:  # November, February, March
        return 'winter_offpeak'
    elif month_number in [12, 1]:   # December, January
        return 'winter_onpeak'
    elif month_number in range(4, 11):  # April to October (inclusive)
        return 'summer'
    else:
        return 'Unknown'  # Fallback, shouldn't happen if months are correct

In [66]:
def sum_therms_by_season(gas_data):
    """Sum the calculated natural gas total therms for each seasonal category."""
    gas_data['season'] = gas_data['month'].apply(categorize_season)
    
    therms_by_season = gas_data.groupby('season')['calculated.natural_gas.total.therms'].sum()
    total_therms = gas_data['calculated.natural_gas.total.therms'].sum()
    
    return therms_by_season, total_therms

seasonal_therms, total_therms = sum_therms_by_season(load_profile_df)

# Output the seasonal therms and total therms
print("Therms by season:")
print(seasonal_therms)
print(f"\nTotal therms: {total_therms}")

Therms by season:
season
summer            153.123827
winter_offpeak    154.477718
winter_onpeak     149.336831
Name: calculated.natural_gas.total.therms, dtype: float64

Total therms: 456.9383758888757


In [67]:
import pandas as pd

def calculate_annual_costs_gas(load_profile_df, rate_plans, baseline_allowances, territory="T"):
    """
    Calculate the annual gas cost based on the load profile and rate plans using seasonal therms.
    
    Args:
        load_profile_df (pd.DataFrame): DataFrame containing the load profile with calculated therms and timestamps.
        rate_plans (dict): Dictionary containing gas rate plan details.
        baseline_allowances (dict): Dictionary containing baseline allowances.
        territory (str): The territory for which the rate applies (default is "T").
    
    Returns:
        float: Total annual gas cost.
    """
    
    # Use the seasonal categorization and sum function
    seasonal_therms, total_therms = sum_therms_by_season(load_profile_df)
    
    # Initialize the total cost
    total_cost = 0.0
    
    # Loop through each season and calculate the cost
    for season, therms_used in seasonal_therms.items():
        # Retrieve the baseline allowance for the season
        baseline = baseline_allowances["PGE"]["G-1"]["territories"][territory][season]
        
        # Determine if usage is within baseline or exceeds it
        if therms_used <= baseline:
            rate = rate_plans["PGE"]["G-1"]["baseline"]["total_charge"]
        else:
            rate = rate_plans["PGE"]["G-1"]["excess"]["total_charge"]
        
        # Calculate the cost for the season
        seasonal_cost = therms_used * rate
        total_cost += seasonal_cost
    
    return total_cost

# Path to the 8760 therms load profile
csv_file_path = f"alameda/{scenario}/{housing_type}/step4_natural_gas_loads.csv"

# Load the therms profile from the CSV
load_profile_df = pd.read_csv(csv_file_path)

# Assuming the total household load in therms is in the column 'calculated.natural_gas.total.therms'
# Example usage: Calculate annual cost for the therms profile in territory "T"
annual_cost = calculate_annual_costs_gas(load_profile_df, rate_plans, baseline_allowances, territory="T")
print(f"Total annual gas cost: ${annual_cost:.2f}")

Total annual gas cost: $1278.39
